In [35]:
import open3d as o3d

In [36]:
def save_point_cloud_as_ply(point_cloud, file_name):
    """Guarda la nube de puntos como un archivo PLY."""
    print(f"Guardando nube de puntos como {file_name}...")
    o3d.io.write_point_cloud(file_name, point_cloud)

def visualize_point_cloud(point_cloud, window_name):
    """Visualiza una nube de puntos en una ventana."""
    print(f"Visualizando nube de puntos en ventana {window_name}...")
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name=window_name)
    vis.add_geometry(point_cloud)
    vis.run()
    vis.destroy_window()



def compare_point_clouds(original_cloud, inlier_cloud, outlier_cloud):
    """
    Compara el número de puntos en las nubes de puntos para verificar la precisión de la segmentación.
    """
    original_count = len(original_cloud.points)
    inlier_count = len(inlier_cloud.points)
    outlier_count = len(outlier_cloud.points)
    
    print(f"Puntos originales: {original_count}")
    print(f"Puntos del plano (inliers): {inlier_count}")
    print(f"Puntos fuera del plano (outliers): {outlier_count}")
    print(f"Total puntos (inliers + outliers): {inlier_count + outlier_count}")

    # Verificar que el conteo coincide
    if original_count == inlier_count + outlier_count:
        print("La comparación es correcta. Los puntos fuera del plano coinciden con la nube original menos los inliers.")
    else:
        print("La comparación no es correcta. Verifica la segmentación y el procesamiento de los datos.")



In [37]:
# Ruta del archivo de la nube de puntos
file_path = "dragon_backdrop\dragonBk1_0.ply"
# Cargar la nube de puntos
print("Cargando la nube de puntos...")
point_cloud = o3d.io.read_point_cloud(file_path)

Cargando la nube de puntos...


In [38]:
# Aplicar filtro de voxel
voxel_size = 0.02
print("Aplicando filtro de voxel para reducir el número de puntos...")
downsampled_point_cloud = point_cloud.voxel_down_sample(voxel_size)

Aplicando filtro de voxel para reducir el número de puntos...


In [39]:
# Segmentación del plano
print("Segmentando el plano utilizando RANSAC...")
plane_model, inliers = downsampled_point_cloud.segment_plane(distance_threshold=0.01, ransac_n=3, num_iterations=1000)
print(f"Modelo del plano: {plane_model}")
print(f"Número de inliers: {len(inliers)}")

Segmentando el plano utilizando RANSAC...
Modelo del plano: [-0.00418686 -0.00153112  0.99999006  0.06234177]
Número de inliers: 122


In [40]:
# Extraer y guardar los puntos del plano y los puntos fuera del plano
inlier_cloud = downsampled_point_cloud.select_by_index(inliers)
outlier_cloud = downsampled_point_cloud.select_by_index(inliers, invert=True)
    

In [41]:
# Guardar las nubes de puntos segmentadas
save_point_cloud_as_ply(inlier_cloud, "inlier_cloud.ply")
save_point_cloud_as_ply(outlier_cloud, "outlier_cloud.ply")

# Visualizar la nube de puntos original
visualize_point_cloud(point_cloud, "Nube de Puntos Original")

Guardando nube de puntos como inlier_cloud.ply...
Guardando nube de puntos como outlier_cloud.ply...
Visualizando nube de puntos en ventana Nube de Puntos Original...


In [45]:
# Visualizar las nubes de puntos guardadas
visualize_point_cloud(inlier_cloud, "Nube de Puntos del Plano")
visualize_point_cloud(outlier_cloud, "Nube de Puntos Fuera del Plano")

Visualizando nube de puntos en ventana Nube de Puntos del Plano...
Visualizando nube de puntos en ventana Nube de Puntos Fuera del Plano...


In [43]:
# Comparar las nubes de puntos
compare_point_clouds(downsampled_point_cloud, inlier_cloud, outlier_cloud)

Puntos originales: 260
Puntos del plano (inliers): 122
Puntos fuera del plano (outliers): 138
Total puntos (inliers + outliers): 260
La comparación es correcta. Los puntos fuera del plano coinciden con la nube original menos los inliers.
